In [1]:
from pyjet import cluster,DTYPE_EP,DTYPE_PTEPM
import numba as nb

In [2]:
import sys
import IPython
#import ROOT
import awkward as ak
import vector
vector.register_awkward()
import os
import glob
import matplotlib.pyplot as plt
import pandas as pd
#import root_pandas as rpd
import uproot3
import uproot
import uproot4
#import awkward as ak
import numpy as np
import math
#timers
import time
#if args.time:
#    start = time.clock()
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

In [3]:
#tree2 = uproot.open("root://polgrid4.in2p3.fr//dpm/in2p3.fr/home/cms/trivcat/store/user/usarkar/VBF_HToInvisible_M125_14TeV_powheg_pythia8_TuneCP5/crab_Multialgo_HGCAL/210930_084934/0000/ntuple_1.root",xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["FloatingpointThresholdDummyHistomaxGenmatchGenclustersntuple/HGCalTriggerNtuple"]
#tree = uproot3.open("root://polgrid4.in2p3.fr//dpm/in2p3.fr/home/cms/trivcat/store/user/usarkar/VBF_HToInvisible_M125_14TeV_powheg_pythia8_TuneCP5/crab_Multialgo_HGCAL/210930_084934/0000/ntuple_1.root",xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["FloatingpointThresholdDummyHistomaxGenmatchGenclustersntuple/HGCalTriggerNtuple"]
tree = uproot3.open("ntuple.root",xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["FloatingpointThresholdDummyHistomaxGenmatchGenclustersntuple/HGCalTriggerNtuple"]
tree2 = uproot.open("ntuple.root",xrootdsource=dict(chunkbytes=1024**3, limitbytes=1024**3))["FloatingpointThresholdDummyHistomaxGenmatchGenclustersntuple/HGCalTriggerNtuple"]
full = ["gen_eta","gen_phi","gen_pt","gen_pdgid","gen_energy","genjet_eta","genjet_phi","genjet_pt","genjet_energy","cl3d_pt","cl3d_energy","cl3d_eta","cl3d_phi"]
branches = ["genjet_eta_*","genjet_phi_*","genjet_pt_*","genjet_energy_*","cl3d_pt","cl3d_energy","cl3d_eta","cl3d_phi"]
branches2 = ["genjet_eta","genjet_phi","genjet_pt","genjet_energy"]

tree.show()

run                        (no streamer)              asdtype('>i4')
event                      (no streamer)              asdtype('>i4')
lumi                       (no streamer)              asdtype('>i4')
gen_n                      (no streamer)              asdtype('>i4')
gen_PUNumInt               (no streamer)              asdtype('>i4')
gen_TrueNumInt             (no streamer)              asdtype('>f4')
vtx_x                      (no streamer)              asdtype('>f4')
vtx_y                      (no streamer)              asdtype('>f4')
vtx_z                      (no streamer)              asdtype('>f4')
gen_eta                    TStreamerSTL               asjagged(asdtype('>f4'), 10)
gen_phi                    TStreamerSTL               asjagged(asdtype('>f4'), 10)
gen_pt                     TStreamerSTL               asjagged(asdtype('>f4'), 10)
gen_energy                 TStreamerSTL               asjagged(asdtype('>f4'), 10)
gen_charge                 TStreamerSTL        

In [4]:
fulldf = tree.pandas.df(full,flatten=False)
df = tree.pandas.df(branches,entrystart=0,entrystop=tree.numentries)
df2 = tree.pandas.df(branches2,entrystart=0,entrystop=tree.numentries)
fulldf["ak4jet_eta"] = fulldf["cl3d_eta"]
fulldf["ak4jet_phi"] = fulldf["cl3d_phi"]
fulldf["ak4jet_pt"] = fulldf["cl3d_pt"]
fulldf["ak4jet_constituents"] = fulldf["cl3d_energy"]
#fulldf

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
exdf = tree2.arrays(["gen_energy","gen_pt", "gen_eta", "gen_phi", "gen_pdgid","gen_status","genjet_energy","genjet_pt", "genjet_eta", "genjet_phi"],library="pd",entry_stop=1)

In [6]:
exdf

(                 gen_energy        gen_pt       gen_eta       gen_phi  \
 entry subentry                                                          
 0     0         7000.000000  0.000000e+00  29756.000000  0.000000e+00   
       1         7000.000000  0.000000e+00 -29756.000000  0.000000e+00   
       2          204.048828  0.000000e+00  22960.048828  0.000000e+00   
       3          525.642578  0.000000e+00 -23281.642578  0.000000e+00   
       4          202.115921  1.330407e+02     -0.612962 -5.321555e-01   
       5          135.296127  3.060044e+00      4.482053  2.686261e+00   
       6          324.524872  1.079432e+02     -1.765010  2.599925e+00   
       7           67.753090  2.206556e+01     -1.787361  2.645332e+00   
       8         1265.509766  0.000000e+00  24021.509766  0.000000e+00   
       9            1.568359  0.000000e+00 -22757.568359  0.000000e+00   
       10        1259.824951  5.961930e+00      6.046475  4.153439e-02   
       11           7.253045  5.961930

In [7]:
for i_event in range(tree.numentries):   
    genDF = df.loc[i_event]
    genjet = df2.loc[i_event,['genjet_pt','genjet_eta','genjet_phi','genjet_energy']]
    #genjet.columns=["pT","eta","phi","energy"]
    cl3d = df.loc[i_event,['cl3d_pt','cl3d_eta','cl3d_phi','cl3d_energy']]
    cl3d.columns=["pT","eta","phi","energy"]
    cl3d = cl3d.assign(mass=0)
    #go dataframe to np array, formatted for input into fastjet    
    cl3dVectors = np.array(cl3d[["pT","eta","phi","energy"]].to_records(index=False).astype([(u'pT', '<f8'), (u'eta', '<f8'), (u'phi', '<f8'), (u'mass', '<f8')]) )    
    clusterVals = cluster(cl3dVectors,R=0.4,algo="antikt")
    _jets = clusterVals.inclusive_jets() 
    ak4jet_eta = []
    ak4jet_phi = []
    ak4jet_pt = []
    ak4jetmatched_pt = []
    constituent = []
    deta = []
    dphi = []
    deltar = []
    deltarmatched = []
    dpt = []
    jet_index = []
    for j,jet in enumerate(_jets):
        ak4jet_pt.append(jet.pt)
        ak4jet_eta.append(jet.eta)
        ak4jet_phi.append(jet.phi)
        constituent.append((len(_jets[j].constituents_array())))
        
    fulldf['ak4jet_eta'][i_event] = ak4jet_eta
    fulldf['ak4jet_phi'][i_event] = ak4jet_phi
    fulldf['ak4jet_pt'][i_event] = ak4jet_pt
    fulldf['ak4jet_constituents'][i_event] = constituent


In [8]:
#list_ak4jet = {key: fulldf[key].values for key in ['ak4jet_pt', 'ak4jet_eta', 'ak4jet_phi']}
#ak4jet = ak.Array(list_ak4jet)
#print(ak4jet)

list_ak4jet_pt = fulldf["ak4jet_pt"].to_numpy()
list_ak4jet_eta = fulldf["ak4jet_eta"].to_numpy()
list_ak4jet_phi = fulldf["ak4jet_phi"].to_numpy()
ak4jet_pt = ak.Array(list_ak4jet_pt)
ak4jet_eta = ak.Array(list_ak4jet_eta)
ak4jet_phi = ak.Array(list_ak4jet_phi)
ak4jet = ak.zip({"pt":ak4jet_pt,"eta":ak4jet_eta,"phi":ak4jet_phi})
#ak4jet

list_genjet_pt = fulldf["genjet_pt"].to_numpy()
list_genjet_eta = fulldf["genjet_eta"].to_numpy()
list_genjet_phi = fulldf["genjet_phi"].to_numpy()
genjet_pt = ak.Array(list_genjet_pt)
genjet_eta = ak.Array(list_genjet_eta)
genjet_phi = ak.Array(list_genjet_phi)
genjet = ak.zip({"pt":genjet_pt,"eta":genjet_eta,"phi":genjet_phi})

list_gen_pt = fulldf["gen_pt"].to_numpy()
list_gen_eta = fulldf["gen_eta"].to_numpy()
list_gen_phi = fulldf["gen_phi"].to_numpy()
list_gen_pdgid = fulldf["gen_pdgid"].to_numpy()
gen_pt = ak.Array(list_gen_pt)
gen_eta = ak.Array(list_gen_eta)
gen_phi = ak.Array(list_gen_phi)
gen_pdgid = ak.Array(list_gen_pdgid)
gen = ak.zip({"pt":gen_pt,"eta":gen_eta,"phi":gen_phi,"pdgid":gen_pdgid})

In [9]:
#genjet = tree2.arrays(
#    ["pt", "eta","phi"],
#    aliases={"pt": "genjet_pt", "eta": "genjet_eta", "phi": "genjet_phi"}
#)

genjets = ak.with_name(genjet, "Momentum3D")
genjets

gen = ak.with_name(gen, "Momentum4D")
gen


gen_genjet = ak.cartesian({"gen": gen, "genjets": genjets}, nested=True)
gen, genjet = ak.unzip(gen_genjet)
gen, genjet



(<MomentumArray4D [[[{pt: 0, ... pdgid: 22}]]] type='5 * var * var * Momentum4D["...'>,
 <MomentumArray3D [[[{pt: 88, ... phi: -1.6}]]] type='5 * var * var * Momentum3D[...'>)

In [10]:
gen.deltaR(genjet)

<Array [[[2.98e+04, 2.98e+04, ... 2.97, 3.05]]] type='5 * var * var * float64'>

In [11]:
deltaR1 = ak.flatten(gen.deltaR(genjet)[abs(gen.pdgid) == 1], axis=None)
import hist as h
h.Hist.new.Reg(100, 0, 20).Double().fill(
    ak.flatten(deltaR1, axis=None)
)

Hist(Regular(100, 0, 20, label='Axis 0'), storage=Double()) # Sum: 5404.0 (6517.0 with flow)

In [12]:
#ak.flatten(gen.deltaR(genjet)[abs(gen.pdgid) == 1 or abs(gen.pdgid) == 2], axis=None)

In [13]:
ak4jets = ak.with_name(ak4jet, "Momentum3D")
ak4jets
reco_gen = ak.cartesian({"ak4jets": ak4jets, "genjets": genjets}, nested=True)
ak4jet, genjet = ak.unzip(reco_gen)
ak4jet, genjet

#reco_gen

(<MomentumArray3D [[[{pt: 72.3, ... phi: -0.683}]]] type='5 * var * var * Momentu...'>,
 <MomentumArray3D [[[{pt: 88, ... phi: -1.6}]]] type='5 * var * var * Momentum3D[...'>)

In [14]:
ak4jet.deltaR(genjet)

#deltapt = 

<Array [[[0.395, 1.18, 1.77, ... 1.65, 1.04]]] type='5 * var * var * float64'>

In [15]:
import hist as h
h.Hist.new.Reg(100, 0, 20).Double().fill(
    ak.flatten(ak4jet.deltaR(genjet), axis=None)
)

Hist(Regular(100, 0, 20, label='Axis 0'), storage=Double()) # Sum: 1468.0

In [16]:
ak.min(ak4jet.deltaR(genjet), axis=-1)

<Array [[0.395, 0.116, 0.598, ... 1.42, 0.76]] type='5 * var * ?float64'>

In [17]:
h.Hist.new.Reg(100, 0, 5).Double().fill(
        ak.flatten(ak.min(ak4jet.deltaR(genjet), axis=-1), axis=None)
)


Hist(Regular(100, 0, 5, label='Axis 0'), storage=Double()) # Sum: 75.0

In [18]:
best = ak.argmin(ak4jet.deltaR(genjet), axis=-1, keepdims=True)
best

<Array [[[0], [18], [6], ... [1], [6], [1]]] type='5 * var * var * ?int64'>

In [19]:
reco_gen[best]

<Array [[[{ak4jets: {pt: 72.3, ... ] type='5 * var * var * ?{"ak4jets": Momentum...'>

In [20]:
ak.flatten(reco_gen[best], axis=-1)[:4].tolist()

ak4jetmatched, genjetmatched = ak.unzip(reco_gen[best])
ak4jetmatched, genjetmatched

(<MomentumArray3D [[[{pt: 72.3, ... phi: -0.683}]]] type='5 * var * var * ?Moment...'>,
 <MomentumArray3D [[[{pt: 88, ... phi: -1.44}]]] type='5 * var * var * ?Momentum3...'>)

In [21]:
#fig, ax0 = plt.subplots(1, 1, figsize=(12,12))

h.Hist.new.Reg(100, 0, 50).Double().fill(
        ak.flatten(ak4jet.pt, axis=None)
)

Hist(Regular(100, 0, 50, label='Axis 0'), storage=Double()) # Sum: 1430.0 (1468.0 with flow)

In [22]:
h.Hist.new.Reg(100, 0, 50).Double().fill(
        ak.flatten(genjet.pt, axis=None)
)

Hist(Regular(100, 0, 50, label='Axis 0'), storage=Double()) # Sum: 1380.0 (1468.0 with flow)